In [1]:
#本文件用以生成训练集和测试集

In [26]:
#通用
from tqdm.notebook import tqdm
#数据处理
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time

In [27]:
class mergesort(): 
    def __init__(self):
        return
    def mergesort(self,nums):
        
        if len(nums)<=1:
            return nums
        left = self.mergesort(nums[:int(len(nums)/2)])
        right = self.mergesort(nums[int(len(nums)/2):])
        result = self.merge(left,right)
        return result

    def merge(self,left,right):
        result = []
        p1, p2 = 0, 0
        while p1 < len(left) and p2 < len(right):
            if left[p1] < right[p2]:
                result.append(left[p1])
                p1 += 1
            else:
                result.append(right[p2])
                p2 += 1
        result += left[p1:len(left)]
        result += right[p2:len(right)]
        return result


In [28]:
train_data = pd.read_csv('C:/ml_data/df/server/train.csv')
test_data = pd.read_csv('C:/ml_data/df/server/evaluation_public.csv')
sub_data = pd.read_csv('C:/ml_data/df/server/submit_example.csv')

In [29]:
#测试集中STATUS 只有 available
#测试集中QUEUE_TYPE 只有 Sql 和 general
#测试集中PLATFORM只有x86

#删除训练集中超出测试范围的数据

In [30]:
train_size_old = len(train_data)
train_data.drop(index = train_data[train_data['STATUS'] != 'available'].index, inplace = True)
train_data.drop(index = train_data[train_data['QUEUE_TYPE'] == 'spark'].index, inplace = True)
train_data.drop(index = train_data[train_data['PLATFORM'] == 'aarch64'].index, inplace = True)
train_size_new = len(train_data)
print('保留了',int(train_size_new/train_size_old*100),'%的数据')

保留了 95 %的数据


In [31]:
train_data.head()

,QUEUE_ID,CU,STATUS,QUEUE_TYPE,PLATFORM,CPU_USAGE,MEM_USAGE,LAUNCHING_JOB_NUMS,RUNNING_JOB_NUMS,SUCCEED_JOB_NUMS,CANCELLED_JOB_NUMS,FAILED_JOB_NUMS,DOTTING_TIME,RESOURCE_TYPE,DISK_USAGE
0,298,16,available,sql,x86_64,0,0,0,0,4,0,0,1582548540000,NaN,NaN
1,298,16,available,sql,x86_64,0,0,0,0,4,0,0,1582548840000,NaN,NaN
2,21671,256,available,sql,x86_64,0,0,0,0,0,0,1,1582549500000,NaN,NaN
3,21671,256,available,sql,x86_64,0,0,0,0,4,0,0,1582549500000,NaN,NaN
4,297,16,available,sql,x86_64,0,0,0,0,7,0,0,1582549500000,NaN,NaN


In [32]:
#这里有一个异常点，按照我的理解，QUEUE_ID和CU应该是一一对应的，但是训练数据中存在一个队列，这个队列有6个CU。
#训练集中的其他队列，测试集中的所有队列，每个队列都是只有一个CU数量。
#目前把训练集的这个队列按照异常点处理，删除处理。
tt = train_data.groupby('QUEUE_ID')
for i in tt:
    print(i[0],np.unique(i[1]['CU']))
print('删除了',len(train_data[train_data['QUEUE_ID'] == 85977].index),'条数据')
train_data.drop(index = train_data[train_data['QUEUE_ID'] == 85977].index, inplace = True)

2 [16]
3 [16]
4 [16]
26 [1024]
27 [512]
36 [16]
233 [16]
281 [64]
287 [16]
291 [1024]
293 [960]
297 [16]
298 [16]
21487 [64]
21671 [256]
21673 [64]
21825 [256]
81221 [16]
82695 [16]
82697 [16]
82929 [64]
83109 [16]
83609 [16]
84151 [16]
84387 [16]
84907 [16]
85101 [64]
85153 [64]
85265 [64]
85267 [64]
85617 [16]
85619 [16]
85693 [16]
85731 [16]
85781 [64]
85915 [64]
85933 [16]
85977 [ 16  32  48  64 112 128]
86865 [64]
86867 [256]
87139 [16]
删除了 8793 条数据


In [33]:
#按照queue_id和时间排序，方便肉眼观察，测试集数据也进行处理，测试集数据的顺序不影响最后提交
train_data = train_data.sort_values(by=['QUEUE_ID','DOTTING_TIME'], ascending=True)
test_data = test_data.sort_values(by=['QUEUE_ID','DOTTING_TIME'], ascending=True)

In [34]:
#检测以某个时间点开始，训练集中是否存在连续的五个时间点，每个时间点间隔5分钟
#使用这个函数不用in检测，会降低时间复杂度
#notice：输入data必须是预先排序好的
def valid_continuous_five(data,timestamp):
    for i in range(1,5):
        time_ = timestamp + 300000*i
        if time_ not in data:
            return False
    return True

def valid_single_one(data,timestamp):
    return timestamp in data

'''
def valid_continuous_five(data,timestamp):
    for i in range(1,5):
        time_ = timestamp + 300000*i
        for j in data:
            if j < time_:
                continue
            elif j == time_:
                break
            else:
                return False
    return True
#检测某个时间戳是否在数据中
#使用这个函数不用in检测，会降低时间复杂度
def valid_single_one(data,timestamp):
    for i in data:
        if i < timestamp:
            continue
        elif i == timestamp:
            return True
        else:
            return False
    return True
'''

'\ndef valid_continuous_five(data,timestamp):\n    for i in range(1,5):\n        time_ = timestamp + 300000*i\n        for j in data:\n            if j < time_:\n                continue\n            elif j == time_:\n                break\n            else:\n                return False\n    return True\n#检测某个时间戳是否在数据中\n#使用这个函数不用in检测，会降低时间复杂度\ndef valid_single_one(data,timestamp):\n    for i in data:\n        if i < timestamp:\n            continue\n        elif i == timestamp:\n            return True\n        else:\n            return False\n    return True\n'

In [35]:
#生成训练集数据

In [36]:
tt = train_data.groupby('QUEUE_ID')

In [37]:
#生成总体的训练数据，如果是机器学习模型需要再分开，单独生成十套数据训练十个模型。神经网络模型需要一套数据即可，但是需要再整理数据。

In [38]:
total_X = {}
first_time = 1
count = 0
for i in tt:
    count += 1
    print('Now dealing {}/41'.format(count))
    queue_data = i[1].sort_values(by='DOTTING_TIME', ascending=True)
    queue_data = queue_data.reset_index(drop = True)
    time_data = [x for x in queue_data['DOTTING_TIME']]
    
    for ii in tqdm(range(0,len(queue_data)-5)):
        if not valid_continuous_five(time_data, time_data[ii]):
            continue
        else:
            X = {}
            for temp_loop in range(5):
                continous_data = queue_data.loc[[ii + temp_loop]]
                X['CU_{}'.format(temp_loop)] = continous_data['CU'].values[0]
                X['QUEUE_TYPE_{}'.format(temp_loop)] = continous_data['QUEUE_TYPE'].values[0]
                X['CPU_USAGE_{}'.format(temp_loop)] = continous_data['CPU_USAGE'].values[0]
                X['MEM_USAGE_{}'.format(temp_loop)] = continous_data['MEM_USAGE'].values[0]
                X['LAUNCHING_JOB_NUMS_{}'.format(temp_loop)] = continous_data['LAUNCHING_JOB_NUMS'].values[0]
                X['RUNNING_JOB_NUMS_{}'.format(temp_loop)] = continous_data['RUNNING_JOB_NUMS'].values[0]
                X['SUCCEED_JOB_NUMS_{}'.format(temp_loop)] = continous_data['SUCCEED_JOB_NUMS'].values[0]
                X['CANCELLED_JOB_NUMS_{}'.format(temp_loop)] = continous_data['CANCELLED_JOB_NUMS'].values[0]
                X['RESOURCE_TYPE_{}'.format(temp_loop)] = continous_data['RESOURCE_TYPE'].values[0]
                X['DISK_USAGE_{}'.format(temp_loop)] = continous_data['DISK_USAGE'].values[0]
            #预测目标值从1开始
            for iii in range(1,6):
                time_stamp = time_data[ii] + 300000 * (4+iii)
                if not valid_single_one(time_data, time_stamp):
                    X['TARGET_CPU_USAGE_{}'.format(iii)] = np.nan
                    X['TARGET_LAUNCHING_JOB_NUMS_{}'.format(iii)] = np.nan
                else:
                    X['TARGET_CPU_USAGE_{}'.format(iii)] = queue_data.loc[queue_data['DOTTING_TIME'] == time_stamp]['CPU_USAGE'].values[0]
                    X['TARGET_LAUNCHING_JOB_NUMS_{}'.format(iii)] = queue_data.loc[queue_data['DOTTING_TIME'] == time_stamp]['LAUNCHING_JOB_NUMS'].values[0]
            XX = pd.DataFrame(X,index = [0])
            
            if first_time:
                first_time = 0
                total_X = XX
            else:
                total_X = pd.concat([total_X,XX],axis = 0)
        
        
        
    

Now dealing 1/41



Now dealing 2/41



Now dealing 3/41



Now dealing 4/41



Now dealing 5/41


...


Now dealing 36/41



Now dealing 37/41



Now dealing 38/41



Now dealing 39/41



Now dealing 40/41


In [39]:
total_X.to_csv('C:/ml_data/df/server/eda_engineering_1.csv', index = False)

In [16]:
#下面的代码用以生成测试集数据
#这里面又有一个坑啊，测试集的特征'QUEUE_ID'没有意义，应该使用'ID'

In [17]:
test_data.head(10)

,ID,QUEUE_ID,CU,STATUS,QUEUE_TYPE,PLATFORM,CPU_USAGE,MEM_USAGE,LAUNCHING_JOB_NUMS,RUNNING_JOB_NUMS,SUCCEED_JOB_NUMS,CANCELLED_JOB_NUMS,FAILED_JOB_NUMS,DOTTING_TIME,RESOURCE_TYPE,DISK_USAGE
5325,1066,2,16,available,sql,x86_64,3,48,0,0,0,0,0,1596587880000,vm,26
5326,1066,2,16,available,sql,x86_64,2,48,0,0,0,0,0,1596588180000,vm,26
5327,1066,2,16,available,sql,x86_64,3,48,0,0,0,0,0,1596588480000,vm,26
5328,1066,2,16,available,sql,x86_64,3,48,0,0,0,0,0,1596588780000,vm,26
5329,1066,2,16,available,sql,x86_64,6,48,0,0,0,0,0,1596589080000,vm,26
12825,2566,2,16,available,sql,x86_64,3,50,0,0,0,0,0,1596654240000,vm,26
12826,2566,2,16,available,sql,x86_64,7,50,0,0,0,0,0,1596654540000,vm,26
12827,2566,2,16,available,sql,x86_64,3,50,0,0,2,0,0,1596654840000,vm,26
12828,2566,2,16,available,sql,x86_64,5,50,0,0,0,0,0,1596655140000,vm,26
12829,2566,2,16,available,sql,x86_64,2,50,0,0,0,0,0,1596655440000,vm,26


In [18]:
tt = test_data.groupby('ID')

In [19]:
test_total_X = {}
first_time = 1
for i in tqdm(tt):
    queue_data = i[1].sort_values(by='DOTTING_TIME', ascending=True)
    queue_data = queue_data.reset_index(drop = True)
    X = {}
    for temp_loop in range(5):
        continous_data = queue_data.loc[[temp_loop]]
        X['CU_{}'.format(temp_loop)] = continous_data['CU'].values[0]
        X['QUEUE_TYPE_{}'.format(temp_loop)] = continous_data['QUEUE_TYPE'].values[0]
        X['CPU_USAGE_{}'.format(temp_loop)] = continous_data['CPU_USAGE'].values[0]
        X['MEM_USAGE_{}'.format(temp_loop)] = continous_data['MEM_USAGE'].values[0]
        X['LAUNCHING_JOB_NUMS_{}'.format(temp_loop)] = continous_data['LAUNCHING_JOB_NUMS'].values[0]
        X['RUNNING_JOB_NUMS_{}'.format(temp_loop)] = continous_data['RUNNING_JOB_NUMS'].values[0]
        X['SUCCEED_JOB_NUMS_{}'.format(temp_loop)] = continous_data['SUCCEED_JOB_NUMS'].values[0]
        X['CANCELLED_JOB_NUMS_{}'.format(temp_loop)] = continous_data['CANCELLED_JOB_NUMS'].values[0]
        X['RESOURCE_TYPE_{}'.format(temp_loop)] = continous_data['RESOURCE_TYPE'].values[0]
        X['DISK_USAGE_{}'.format(temp_loop)] = continous_data['DISK_USAGE'].values[0]
    X = pd.DataFrame(X,index = [0])
    if first_time:
        first_time = 0
        test_total_X = X
    else:
        test_total_X = pd.concat([test_total_X,X],axis = 0)
        
    

In [20]:
test_total_X.to_csv('C:/ml_data/df/server/eda_engineering_1_test_data.csv', index = False)